In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./resources', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./resources', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.177
[1,  4000] loss: 1.826
[1,  6000] loss: 1.685
[1,  8000] loss: 1.615
[1, 10000] loss: 1.574
[1, 12000] loss: 1.509
[2,  2000] loss: 1.483
[2,  4000] loss: 1.422
[2,  6000] loss: 1.413
[2,  8000] loss: 1.345
[2, 10000] loss: 1.332
[2, 12000] loss: 1.325
Finished Training


In [7]:
from clearbox_wrapper.PytorchWrapper import PytorchWrapper

In [8]:
class MyModel(PytorchWrapper):
    def __init__(self, model):
        super().__init__(model)
        self.transformer = transform
    
    def predict(self, path):
        from PIL import Image

        img = Image.open(path)
   
        img_tensor = self.transformer(img).float()
        img_tensor = img_tensor.unsqueeze_(0)

        fc_out = self.model(img_tensor)

        output = fc_out.detach().numpy()
        return output

In [9]:
MyModel(net).dump('torch_cifar.model')

In [10]:
foo = PytorchWrapper.load('torch_cifar.model')

In [11]:
foo.predict("./resources/cifar-10-example.jpg")

array([[ 3.3223448,  8.756701 , -1.9293946, -3.2061927, -3.882034 ,
        -4.328064 , -2.5199485, -3.4717033,  2.949646 ,  5.6828027]],
      dtype=float32)